In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [2]:
template = "https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search=data%20engineer%20"

In [3]:
def get_url(location,page,position):
    """Generate a. url from position and location"""
    #template = "https://jobs.apple.com/en-us/search?search={}&sort=relevance&location={}"
    template = "https://jobs.apple.com/en-us/search?location={}&page={}&sort=newest&search={}"
    url = template.format(location,page,position)
    return url

In [4]:
url = get_url('united-states-USA',1,'"Data Engineer"')

In [5]:
response = requests.get(url)
response

<Response [200]>

In [6]:
soup = BeautifulSoup(response.text,'html.parser')
soup

<!DOCTYPE html>

<html lang="en-US" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=10;IE=11;IE=edge" httpequiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<script src="/static/js/analytics/v4/packager-20201111A-JOBS.launch-20201111F-prod.js"></script>
<link href="/static/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<!--<link rel="stylesheet" type="text/css" href="/search/filemaker/filemaker.css">-->
<title data-react-helmet="true">United States - Jobs - Careers at Apple</title>
<meta content="noindex,follow" data-react-helmet="true" name="robots"><meta content="Explore all United States jobs at Apple. Create a profile and apply today." data-react-helmet="true" name="description"/><meta content="website" data-react-helmet="true" property="og:type"/><meta content="United States - Jobs - Careers at Apple" data-react-helmet="true" property="og:title"/><meta content="https://jobs.apple

In [7]:
#total = soup.find_all('nav','pagination')[0].ul.find_all('li','pagination__mid')[0].find_all('span','pageNumber')
#total = total[1].text
#total = int(total)
#total

In [7]:
def get_total_results_count(soup):
    results = soup.find('h2',{"id": "resultCount"}).span.text


    num = 0
    numchar = ""
    for char in results:
        if char.isnumeric():
            numchar+=char
    results = int(numchar)
    return results

In [8]:
num_total_jobs = get_total_results_count(soup)
num_total_jobs

80

store responses 

In [9]:
urls = []
job_id_list_of_lists = []

In [10]:
import os
from ast import literal_eval

def check_if_job_id_exists_on_file(job_id):
    the_job_id_file = "/Users/williamsalinas/Documents/Project/job_id_list.txt"

    if os.stat(the_job_id_file).st_size != 0:
        with open(the_job_id_file) as f:
            content = f.read().replace('\n','')
        job_id_array_from_file=literal_eval(content)

        
        array_of_list_from_file = []
        for i in job_id_array_from_file:
            for job_id_from_file in i:
                array_of_list_from_file.append(job_id_from_file)
                
        if job_id not in array_of_list_from_file:
            return job_id

In [12]:
def job_id_list_func(soup):
    job_id_list = []
    jobs_list = soup.find_all('td','table-col-1')
    for job_url in jobs_list:
        job_id = job_url.a.get('id')
        #NEW STUFF 06/05/22
        id_exists = check_if_job_id_exists_on_file(job_id)
        if id_exists != None:
                #THIS APPEND ALREADY EXISTED
            job_id_list.append(job_id)
        ##END OF NEW STUFF
    return job_id_list

In [13]:
def get_total_pages():
    url = get_url('united-states-USA',1,'Data Engineer')
    soup = BeautifulSoup(response.text,'html.parser')
    total = soup.find_all('nav','pagination')[0].ul.find_all('li','pagination__mid')[0].find_all('span','pageNumber')
    total = total[1].text
    total = int(total)
    return total

In [14]:
total = get_total_pages()

In [15]:
total

4

In [16]:
total = get_total_pages()
#total = 2
for i in range(total):
    url = get_url('united-states-USA',i+1,'"Data Engineer"')
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    job_id_list=job_id_list_func(soup)
    
    job_id_list_of_lists.append(job_id_list)
    urls.append(url)

In [17]:
job_id_list_of_lists

[['jotTitle_PIPE-200373024',
  'jotTitle_PIPE-200387601',
  'jotTitle_PIPE-200385831',
  'jotTitle_REQ-200382325',
  'jotTitle_REQ-200291914',
  'jotTitle_PIPE-200385829',
  'jotTitle_REQ-200312775',
  'jotTitle_REQ-200363686',
  'jotTitle_REQ-200375629',
  'jotTitle_REQ-200353167',
  'jotTitle_REQ-200383422',
  'jotTitle_REQ-200383383',
  'jotTitle_REQ-200383412',
  'jotTitle_REQ-200383416'],
 ['jotTitle_REQ-200364224',
  'jotTitle_REQ-200374992',
  'jotTitle_REQ-200375054',
  'jotTitle_PIPE-200377893',
  'jotTitle_PIPE-200374125',
  'jotTitle_REQ-200360178',
  'jotTitle_REQ-200372173'],
 ['jotTitle_REQ-200337928'],
 []]

In [18]:
urls

['https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search="Data Engineer"',
 'https://jobs.apple.com/en-us/search?location=united-states-USA&page=2&sort=newest&search="Data Engineer"',
 'https://jobs.apple.com/en-us/search?location=united-states-USA&page=3&sort=newest&search="Data Engineer"',
 'https://jobs.apple.com/en-us/search?location=united-states-USA&page=4&sort=newest&search="Data Engineer"']

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [20]:
def open_job_links(url,job_id):
    detail_job = ""
    driver = webdriver.safari.webdriver.WebDriver(quiet=False)
    driver.get(url);
    button = driver.find_element(By.ID,job_id)
    # clicking on the button
    if button != None:
        button.click()
        sleep(5)
        html_from_page = driver.page_source
        detail_job = BeautifulSoup(html_from_page, 'html.parser')
    return detail_job

In [21]:
detail_job_array = []

In [22]:
num = len(urls)
for i in range(num):
    for job_id in job_id_list_of_lists[i]:
        print(urls[i],job_id)
        detail = open_job_links(urls[i],job_id)
        detail_job_array.append(detail)

https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search="Data Engineer" jotTitle_PIPE-200373024
https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search="Data Engineer" jotTitle_PIPE-200387601
https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search="Data Engineer" jotTitle_PIPE-200385831
https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search="Data Engineer" jotTitle_REQ-200382325
https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search="Data Engineer" jotTitle_REQ-200291914
https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search="Data Engineer" jotTitle_PIPE-200385829
https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search="Data Engineer" jotTitle_REQ-200312775
https://jobs.apple.com/en-us/search?location=united-states-USA&page=1&sort=newest&search="Data Engin

In [23]:
len(detail_job_array)

22

In [24]:
list_of_job_titles = []
for detail_job in detail_job_array:
    job_details_list_but_get_only_element = detail_job.find_all('div','job-details')[0]
    job_detail = job_details_list_but_get_only_element
    job_title = job_detail.h1.text
    list_of_job_titles.append(job_title)
list_of_job_titles

['Data Engineer, Maps Eval Metrics',
 'Software Data Engineer - Analytics',
 'Big Data Engineer - Wallet & Apple Pay - Austin',
 'Senior Health Research Data Engineer',
 'Data Engineer - Home Services Intelligence',
 'Big Data Engineer - Wallet & Apple Pay - Raleigh',
 'Senior Data Engineer, Weather Team',
 'Sr. Data Engineer - Infrastructure, Apps Team',
 'Sr. Data Engineer - Apps Team',
 'Machine Learning Data Engineer - Strategic Data Solutions',
 'Sr. Data Engineer, Apple Cloud Services (Multiple US Locations)',
 'Sr. Data Engineer, Apple Cloud Services (Multiple US Locations)',
 'Sr. Data Engineer, Apple Cloud Services (Multiple US Locations)',
 'Sr. Data Engineer, Apple Cloud Services (Multiple US Locations)',
 'Data Engineer',
 'Customer Analytics - Software Engineer (Spark, Java, Python, SQL)',
 'AI/ML - Data Engineer - Siri and Language Technologies',
 'AI/ML - Data Engineer, Siri and Language Technologies',
 'AI/ML - Data Engineer - Siri and Language Technologies',
 'AI/ML - 

In [25]:
def job_location_func(job_detail):
    job_location = job_detail.find('div','jd__header--info')
    job_location_list = job_location.find_all('span')
    job_location = ""
    for span in job_location_list:
        job_location += span.text
    return job_location

In [26]:
list_of_job_locations = []
for detail_job in detail_job_array:
    job_details_list_but_get_only_element = detail_job.find_all('div','job-details')[0]
    job_detail = job_details_list_but_get_only_element
    job_location = job_location_func(job_detail)
    list_of_job_locations.append(job_location)
list_of_job_locations

['Santa Clara Valley (Cupertino), California, United States',
 'Santa Clara Valley (Cupertino), California, United States',
 'Austin, Texas, United States',
 'Santa Clara Valley (Cupertino), California, United States',
 'San Diego, California, United States',
 'Raleigh, North Carolina, United States',
 'Cambridge, Massachusetts, United States',
 'San Diego, California, United States',
 'San Diego, California, United States',
 'Austin, Texas, United States',
 'Santa Clara Valley (Cupertino), California, United States',
 'Santa Clara Valley (Cupertino), California, United States',
 'Santa Clara Valley (Cupertino), California, United States',
 'Santa Clara Valley (Cupertino), California, United States',
 'United States',
 'Austin, Texas, United States',
 'Seattle, Washington, United States',
 'Seattle, Washington, United States',
 'Seattle, Washington, United States',
 'Seattle, Washington, United States',
 'Cambridge, Massachusetts, United States',
 'Austin, Texas, United States']

In [27]:
def job_qualifictions_func(job_detail):
    test = job_detail.find_all('div', recursive=False)
    test = test[1]
    test2 = test.find_all('div','jd__row--main jd__summary')[0]
    test3 = test2.ul.find_all('span')
    key_qualifications = []
    for span in test3:
        key_qualifications.append(span.text)
    return key_qualifications

In [28]:
list_of_list_of_job_qualifications = []
for detail_job in detail_job_array:
    job_details_list_but_get_only_element = detail_job.find_all('div','job-details')[0]
    job_detail = job_details_list_but_get_only_element
    list_of_job_qualifications = job_qualifictions_func(job_detail)
    list_of_list_of_job_qualifications.append(list_of_job_qualifications)
list_of_list_of_job_qualifications

[['Excellent programming skills, preferably in Python.',
  'Experience in a big data processing framework such as Spark.',
  'Background includes at least one database system (e.g., PostgreSQL).',
  'Strong communication skills and be adaptive for changing requirements and tech stack.',
  'Nice-to-have Experience',
  'Prior industry experience in developing data application at web scale, but will consider talented and enthusiastic new graduates with relevant internships or coursework.',
  'Basic knowledge in Scala or Java for Spark data pipelines.',
  'Experience in developing user-friendly libraries to automate analytic tasks at scale.',
  'Familiar with the software ecosystem for big data analytics and business intelligence.',
  'Basic knowledge in backend web development such as Flask, FastAPI.',
  'Experience with cloud computing environment such as AWS or GCP.'],
 ['-  Self-starter who is dedicated and demonstrates creative and critical thinking abilities',
  '-  Experience in dev

In [29]:
def job_date_posted_func(job_detail):
    test = job_detail.find_all('div', recursive=False)
    test = test[1]
    date_posted = test.find_all('div','jd__row--sidebar jd__summary--sidebar')[0]
    date_posted = date_posted.time.text
    return date_posted

In [30]:
list_of_job_date_posted = []
for detail_job in detail_job_array:
    job_details_list_but_get_only_element = detail_job.find_all('div','job-details')[0]
    job_detail = job_details_list_but_get_only_element
    date_posted = job_date_posted_func(job_detail)
    list_of_job_date_posted.append(date_posted)
list_of_job_date_posted

['Jun 3, 2022',
 'Jun 3, 2022',
 'Jun 2, 2022',
 'May 31, 2022',
 'May 31, 2022',
 'May 27, 2022',
 'May 27, 2022',
 'May 26, 2022',
 'May 26, 2022',
 'May 26, 2022',
 'May 25, 2022',
 'May 25, 2022',
 'May 25, 2022',
 'May 25, 2022',
 'May 20, 2022',
 'May 19, 2022',
 'May 17, 2022',
 'May 10, 2022',
 'May 4, 2022',
 'May 3, 2022',
 'Apr 28, 2022',
 'Feb 21, 2022']

In [31]:
def job_summary_func(job_detail):
    test = job_detail.find_all('div', recursive=False)
    test = test[1]
    test4 = test.find_all('div','jd__row--main jd__summary--main')[0]
    job_summary = test4.text
    return job_summary

In [32]:
list_of_job_summary = []
for detail_job in detail_job_array:
    job_details_list_but_get_only_element = detail_job.find_all('div','job-details')[0]
    job_detail = job_details_list_but_get_only_element
    job_summary = job_summary_func(job_detail)
    list_of_job_summary.append(job_summary)
list_of_job_summary

['Apple Maps is changing, and data is in the driverʼs seat. Our mission is to build the best map in the world. We are looking for a talented software/data engineer to join our cross-functional data science team, focusing on the deep evaluation of Maps services and features. We work directly with the Engineering and Product teams responsible for quantifying, measuring, and monitoring usage and user experiences across Maps platform. Our day-to-day work crosses many functional areas, including experiment design, AB testing, exploratory data analysis, generative analytics, and more. This role is great for a hybrid data scientist / data engineer, or a software engineer who is looking to have a big impact while also having space to learn more about data science.',
 'Are you a fearless thinker? Do you love the challenge of advancing a widely accepted technology? In the Wireless Software group, you’ll be responsible for bringing groundbreaking wireless connectivity to the world through Wi-Fi, 

In [33]:
def job_description_func(job_detail):
    test = job_detail.find_all('div', recursive=False)
    test = test[1]
    test5 = test.find_all('div','jd__row--main jd__summary--main')[1]
    job_description = test5.text
    return job_description

In [34]:
list_of_job_description = []
for detail_job in detail_job_array:
    job_details_list_but_get_only_element = detail_job.find_all('div','job-details')[0]
    job_detail = job_details_list_but_get_only_element
    job_description = job_description_func(job_detail)
    list_of_job_description.append(job_description)
list_of_job_description

["You'll build data products and tools to support collaborative decision-making with our partner teams in engineering and product management. You're willing to learn fast, and adapt well to changing requirements and new technology stacks. Example projects include: building and managing high-quality data tables, pipelines and APIs from various sources including both big data at web scale as well as small curated data sets. Developing libraries to automate analysis tasks and enable data scientists to extract insight from data in scale. Identifying optimization opportunities (e.g. improving query speed, reducing memory or storage) in our data systems, and implementing those enhancements. Integrating various tools and services and provide uniform and easy access to data scientists and other partner teams.",
 'This Role provides analytics for various internal programs and prototypes to support organization in data informed decisions. This role will require working with multiple large data s

In [35]:
def job_education_and_experience_func(job_detail):
    test = job_detail.find_all('div', recursive=False)
    test = test[1]
    job_education_experience = []

    test6 = test.find('div', {"id": "accordion_education&experience"})
    #if type(test6.div.span) != "NoneType":    
    job_education_experience = test6.span
    
    if job_education_experience is None:
        job_education_experience = ""
    else:
        job_education_experience = job_education_experience.text

    return job_education_experience

In [36]:
list_of_job_education_experience = []
for detail_job in detail_job_array:
    job_details_list_but_get_only_element = detail_job.find_all('div','job-details')[0]
    job_detail = job_details_list_but_get_only_element
    job_education_experience = job_education_and_experience_func(job_detail)
    job_education_experience
    list_of_job_education_experience.append(job_education_experience)

In [37]:
list_of_job_education_experience

['Bachelor or advanced degrees in Computer Science, Statistics, Physics, Operations Research, or similar quantitative domain.',
 'MS or BS in CS, Engineering, Math, Statistics, or a related field or equivalent practical experience in data engineering.',
 'Minimum of bachelor’s degree, preferably in Computer Science, Information Technology or EE, or relevant proven experience is preferred',
 'BS degree in Computer Science or related field, or equivalent work experience.',
 '5+ years relevant industry experience in Data Engineering, Data Analysis & Analytics, Artificial Intelligence, Machine Learning, or related technical field.',
 'Minimum of bachelor’s degree, preferably in Computer Science, Information Technology or EE, or relevant proven experience is preferred',
 'BS/MS in CS CE or equivalent technical discipline.',
 'BS in Computer Science or Equivalent.',
 'BS in Computer Science or related field, or equivalent work experience.',
 'BS/MS/Ph.D. in CS, Engineering, Math, Physics, ot

In [38]:
def job_additional_qualifications_func(job_detail):
    test = job_detail.find_all('div', recursive=False)
    test = test[1]
    additinal_qualifications = []
    #check_if_size_greater_1 = len(test.find_all('div','jd__row--main jd__summary'))
    #accordion_additionalrequirements
    test7 = test.find('div', {"id": "accordion_additionalrequirements"})
    if test7 is None:
        additinal_qualifications = []
    else:
        test7 = test.find('div', {"id": "accordion_additionalrequirements"})
        additinal_qualifications_url = test7.ul
        
        if additinal_qualifications_url is None:
            additinal_qualifications = []
        else:
            additinal_qualifications_2 = test7.ul.find_all('span')
            get_spans_from_additional_spans = additinal_qualifications_2

            for span in get_spans_from_additional_spans:
                additinal_qualifications.append(span.text)
            
    return additinal_qualifications

In [39]:
list_of_list_of_additional_qualifications = []
for detail_job in detail_job_array:
    job_details_list_but_get_only_element = detail_job.find_all('div','job-details')[0]
    job_detail = job_details_list_but_get_only_element
    additinal_qualifications = job_additional_qualifications_func(job_detail)
    list_of_list_of_additional_qualifications.append(additinal_qualifications)

In [40]:
list_of_list_of_additional_qualifications

[[],
 ['Understanding in big data pipelines using technologies like Kafka, Flume, or Storm Experience with large scale data warehousing, mining or analytic systems Machine learning is a plus……'],
 [],
 [],
 [],
 [],
 ['Preferred: machine learning exposure like neural networks',
  'Preferred: prior exposure to making infrastructure decisions'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['-  Knowledge of Database(Relational) Systems',
  '-  Additional understanding on Data Security is a plus'],
 [],
 ['You have excellent written and verbal communication skills.',
  'You are Tenacious, Relentless, and Determined',
  'You are curious: always learning new technologies, rapidly synthesizing new information, and understanding “the why” before “the what.”',
  'You are self-directed and capable of operating amid ambiguity.',
  'You are poised and display excellent judgment in prioritizing across difficult tradeoffs.',
  'You are pragmatic: not letting “the perfect” be the enemy of “the good.”',
  'Yo

STORING THEM IN A CSV

In [41]:
import pandas as pd

In [42]:
df = pd.DataFrame({'job_title': list_of_job_titles,
                   'job_location': list_of_job_locations,
                   'job_date_posted': list_of_job_date_posted,
                  'job_qualifications':list_of_list_of_job_qualifications,
                  'job_summary': list_of_job_summary,
                  'job_description':list_of_job_description,
                  'job_education_experience':list_of_job_education_experience,
                  'job_additional_qualifications':list_of_list_of_additional_qualifications})

In [43]:
from IPython.display import display
display(df.style.set_properties(**{
        'width': '230px',
        'max-width': '230px'
    }))

,job_title,job_location,job_date_posted,job_qualifications,job_summary,job_description,job_education_experience,job_additional_qualifications
0,"Data Engineer, Maps Eval Metrics","Santa Clara Valley (Cupertino), California, United States","Jun 3, 2022","['Excellent programming skills, preferably in Python.', 'Experience in a big data processing framework such as Spark.', 'Background includes at least one database system (e.g., PostgreSQL).', 'Strong communication skills and be adaptive for changing requirements and tech stack.', 'Nice-to-have Experience', 'Prior industry experience in developing data application at web scale, but will consider talented and enthusiastic new graduates with relevant internships or coursework.', 'Basic knowledge in Scala or Java for Spark data pipelines.', 'Experience in developing user-friendly libraries to automate analytic tasks at scale.', 'Familiar with the software ecosystem for big data analytics and business intelligence.', 'Basic knowledge in backend web development such as Flask, FastAPI.', 'Experience with cloud computing environment such as AWS or GCP.']","Apple Maps is changing, and data is in the driverʼs seat. Our mission is to build the best map in the world. We are looking for a talented software/data engineer to join our cross-functional data science team, focusing on the deep evaluation of Maps services and features. We work directly with the Engineering and Product teams responsible for quantifying, measuring, and monitoring usage and user experiences across Maps platform. Our day-to-day work crosses many functional areas, including experiment design, AB testing, exploratory data analysis, generative analytics, and more. This role is great for a hybrid data scientist / data engineer, or a software engineer who is looking to have a big impact while also having space to learn more about data science.","You'll build data products and tools to support collaborative decision-making with our partner teams in engineering and product management. You're willing to learn fast, and adapt well to changing requirements and new technology stacks. Example projects include: building and managing high-quality data tables, pipelines and APIs from various sources including both big data at web scale as well as small curated data sets. Developing libraries to automate analysis tasks and enable data scientists to extract insight from data in scale. Identifying optimization opportunities (e.g. improving query speed, reducing memory or storage) in our data systems, and implementing those enhancements. Integrating various tools and services and provide uniform and easy access to data scientists and other partner teams.","Bachelor or advanced degrees in Computer Science, Statistics, Physics, Operations Research, or similar quantitative domain.",[]
1,Software Data Engineer - Analytics,"Santa Clara Valley (Cupertino), California, United States","Jun 3, 2022","['- Self-starter who is dedicated and demonstrates creative and critical thinking abilities', '- Experience in developing product features, functional specifications, and development schedules, represent team and technology', '- Experience in data modeling and data engineering and analytics experience', '- Experience in data management and automation on Spark, Hadoop, and HDFS environments', '- Experience handling data in relational databases and developing ETL pipelines', '- Competency in Java, Scala, Python or similar object-oriented language', '- Confidence with SQL databases and NoSQL databases like Cassandra', '- Experience in Tableau or Splunk or ELK Stack or Airflow', '- Ability to independently learn new technologies', '- Excellent debugging, critical thinking, and interpersonal skills', '- Attention to detail']","Are you a fearless thinker? Do you love the challenge of advancing a widely accepted technology? In the Wireless Software group, you’ll be responsible for bringing groundbreaking wireless connectivity to the world through Wi-Fi, Bl

In [56]:
df.to_csv('test_delta.csv',index=False)

In [67]:
df_old = pd.read_csv('test_merge.csv')

In [68]:
from IPython.display import display
display(df_old.style.set_properties(**{
        'width': '230px',
        'max-width': '230px'
    }))

,job_title,job_location,job_date_posted,job_qualifications,job_summary,job_description,job_education_experience,job_additional_qualifications
0,Lead Data Engineer- MSI,"Santa Clara Valley (Cupertino), California, United States","May 2, 2022","['5+ years of work experience in Java, Scala, Kotlin Python, Elixir, Rust, or equivalent programming language', 'Software engineering or data engineering experience through the full software development lifecycle', 'Deep SQL expertise, data modeling, and data governance in relational databases', 'Working with multiple data storage and analysis toolsets—including current distributed technologies: Druid, Snowflake, SparkSQL, Presto, Hive, Kafka, Redshift or analogous architectures a plus.', 'Experience using ETL/ELT tools and technologies: Reltio, Stibo, Informatica, Apache Atlas, Linkedin DataHub, Lyft Amundsen, Netflix Metacat or Uber Databook or DBT a plus', 'Solid grasp of using cloud native solutions and Data Security on Cloud based applications', 'Building data streaming solutions using Apache Spark (both batch and streaming), Storm/Flink/Flume preferred as well', 'Experience in handling architectural and design considerations such as performance, scalability, reusability and flexibility issues', 'Envision and expertly express the benefits and constraints of technology solutions to technology partners, business partners, and team members', 'Active contributions to open source projects is a plus']","Location: Cupertino, CA Are you a passionate engineer with a product mindset? Do you want to help build a data mesh from the ground up? MSI is looking for a Lead Data Mesh engineer to help enable Apple to build ground breaking products! As our Lead Data Mesh Engineer, you are an integral part of a small data-centric team driving large-scale data infrastructure and processes development, implementation, and improvement. Our organization thrives on collaborative partnerships. Join and play a key role in developing and driving the adoption of Data Mesh and data-centric micro-services. Apple's Manufacturing Systems and Infrastructure (MSI) team is responsible for capturing, consolidating and tracking all manufacturing data for Apple’s products and modules worldwide. Our tools enable teams to confidently use data to shape the next generation of product manufacturing at Apple. We seek a practitioner with experience building large-scale data platforms, analytic tools, and solutions. If you are passionate about making data easily accessible, trusted, and available across the entire business at scale, we'd love to hear from you.","We are evolving our core data platform including: - How data is collected, moved, stored, and processed - What data is collected and where it is stored - How to apply machine learning across large datasets As a part of our team, you’ll be responsible for: - Driving design and development of data models - Implementation and delivery of Enterprise Data Mesh solution for interoperability among various data sets - Innovation: you'll explore data-centric platform technologies and implement the technologies that you recommend - Spotting and implementing optimizations across data usage from multidisciplinary teams - Ensuring data health & quality through monitoring and alerting tools - Planning robust systems that gracefully recover from outages and system issues - Provide support for both Analytics and Operational Platforms","B.S., M.S. or Ph.D. in Computer Science or equivalent experience. Apple is an Equal Opportunity Employer that is committed to inclusion and diversity. We also take affirmative action to offer employment and advancement opportunities to all applicants, including minorities, women, protected veterans, and individuals with disabilities. Apple will not discriminate or retaliate against applicants who inquire about, disclose, or discuss their compensation or that of other applicants. We will ensure that individuals with disabilities are provided reasonable accom

In [69]:
data = [df,df_old]
df2 = pd.concat(data)

In [71]:
from IPython.display import display
display(df2.reset_index(drop=True).style.set_properties(**{
        'width': '230px',
        'max-width': '230px'
    }))

,job_title,job_location,job_date_posted,job_qualifications,job_summary,job_description,job_education_experience,job_additional_qualifications
0,"Data Engineer, Maps Eval Metrics","Santa Clara Valley (Cupertino), California, United States","Jun 3, 2022","['Excellent programming skills, preferably in Python.', 'Experience in a big data processing framework such as Spark.', 'Background includes at least one database system (e.g., PostgreSQL).', 'Strong communication skills and be adaptive for changing requirements and tech stack.', 'Nice-to-have Experience', 'Prior industry experience in developing data application at web scale, but will consider talented and enthusiastic new graduates with relevant internships or coursework.', 'Basic knowledge in Scala or Java for Spark data pipelines.', 'Experience in developing user-friendly libraries to automate analytic tasks at scale.', 'Familiar with the software ecosystem for big data analytics and business intelligence.', 'Basic knowledge in backend web development such as Flask, FastAPI.', 'Experience with cloud computing environment such as AWS or GCP.']","Apple Maps is changing, and data is in the driverʼs seat. Our mission is to build the best map in the world. We are looking for a talented software/data engineer to join our cross-functional data science team, focusing on the deep evaluation of Maps services and features. We work directly with the Engineering and Product teams responsible for quantifying, measuring, and monitoring usage and user experiences across Maps platform. Our day-to-day work crosses many functional areas, including experiment design, AB testing, exploratory data analysis, generative analytics, and more. This role is great for a hybrid data scientist / data engineer, or a software engineer who is looking to have a big impact while also having space to learn more about data science.","You'll build data products and tools to support collaborative decision-making with our partner teams in engineering and product management. You're willing to learn fast, and adapt well to changing requirements and new technology stacks. Example projects include: building and managing high-quality data tables, pipelines and APIs from various sources including both big data at web scale as well as small curated data sets. Developing libraries to automate analysis tasks and enable data scientists to extract insight from data in scale. Identifying optimization opportunities (e.g. improving query speed, reducing memory or storage) in our data systems, and implementing those enhancements. Integrating various tools and services and provide uniform and easy access to data scientists and other partner teams.","Bachelor or advanced degrees in Computer Science, Statistics, Physics, Operations Research, or similar quantitative domain.",[]
1,Software Data Engineer - Analytics,"Santa Clara Valley (Cupertino), California, United States","Jun 3, 2022","['- Self-starter who is dedicated and demonstrates creative and critical thinking abilities', '- Experience in developing product features, functional specifications, and development schedules, represent team and technology', '- Experience in data modeling and data engineering and analytics experience', '- Experience in data management and automation on Spark, Hadoop, and HDFS environments', '- Experience handling data in relational databases and developing ETL pipelines', '- Competency in Java, Scala, Python or similar object-oriented language', '- Confidence with SQL databases and NoSQL databases like Cassandra', '- Experience in Tableau or Splunk or ELK Stack or Airflow', '- Ability to independently learn new technologies', '- Excellent debugging, critical thinking, and interpersonal skills', '- Attention to detail']","Are you a fearless thinker? Do you love the challenge of advancing a widely accepted technology? In the Wireless Software group, you’ll be responsible for bringing groundbreaking wireless connectivity to the world through Wi-Fi, Bl

In [73]:
df2.to_csv('test_merge.csv',index=False)

In [ ]:
#OPEN TO FIX DATE INTO A PROPER DATE FORMAT IN ORDER TO LOAD IT TO THE DATE DATATYPE IN REDSHIFT

In [ ]:
df_merge = pd.read_csv('test_merge.csv')

In [ ]:
from IPython.display import display
display(df_merge.style.set_properties(**{
        'width': '230px',
        'max-width': '230px'
    }))

In [ ]:
python_string_format = "%Y/%m/%d"

In [ ]:
#convert the column in date format to apply strftime
df_merge['job_date_posted'] = pd.to_datetime(df_merge['job_date_posted'])

In [ ]:
#apply desired date format
df_merge['job_date_posted'].dt.strftime(python_string_format)

In [ ]:
#NEEDED TO GET RID OF THE TIME (HOUR,MINITE,SECOND) AND JUST KEEP THE DATE
df_merge['job_date_posted'] = df_merge['job_date_posted'].dt.date

In [ ]:
from IPython.display import display
display(df_merge.style.set_properties(**{
        'width': '230px',
        'max-width': '230px'
    }))

In [ ]:
df_merge.to_csv('test_redshift.csv',index=False)